# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [130]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [131]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000, eps=0.000001):
        self.C = C # regularization constant
        self.eps = eps

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return 1.0 * sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return (self.loss(x+self.eps, answer) - self.loss(x, answer))/self.eps

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            self.w = self.w - step * (self.der_loss(x, self.a(x)) - self.der_reg())
            
            # w_0 update
            self.w0 = self.w0 - step * self.der_loss(x, self.a(x))

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [132]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[1.31728935 1.89346354 1.67474317 1.54140771 1.09876787 1.09874375
 1.00083284 1.80892537 1.54386424 1.65082181 0.96333372 1.91265908
 1.77519187 1.15508834 1.1245742  1.12615374 1.24699147 1.46750566
 1.37469425 1.23397837] -0.4849406446224618


In [133]:
predictions = model.predict(X_test)

In [134]:
print(predictions)

[0 0 1 ... 0 1 0]


In [135]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [136]:
print(len(predictions), sum(predictions))

2000 922


In [137]:
print(accuracy_score(y_test, predictions))

0.5405


In [138]:
accuracy = []
C = 1
while C <= 10000:
    model = MySVM(C)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, predictions))
    C *= 5

In [139]:
accuracy

[0.5485, 0.5485, 0.549, 0.5485, 0.5255, 0.5405]

## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [150]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(C = 10000, verbose=True)

In [151]:
SVC.fit(X_train, y_train)

[LibLinear]

C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=10000, max_iter=1000000, verbose=True)

In [149]:
pred = SVC.predict(X_test)

In [143]:
accuracy_score(y_test, pred)

0.741

In [146]:
C = 1
SVC_accuracy = []
while C <= 10000:
    SVC = LinearSVC(C=C, max_iter = 10000)
    SVC.fit(X_train, y_train)
    pred = SVC.predict(X_test)
    SVC_accuracy.append(accuracy_score(y_test, pred))
    C *= 5

C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Oganes\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of itera